## Sentiment Analysis for Online Reviews

In [17]:
# libraries to import
import string
import math
import pandas as pd
import numpy as np
import nltk as nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer  

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\evatr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\evatr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\evatr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### a) Downloading, reading and analyzing datasets

In [2]:
# load data in the right> format according to readme files
yelp=pd.read_csv("sentiment_labelled_sentences\yelp_labelled.txt",delimiter="\t", names=["Sentence", "Label"])
imdb=pd.read_csv("sentiment_labelled_sentences\imdb_labelled.txt",delimiter="\t", names=["Sentence", "Label"])
amazon=pd.read_csv("sentiment_labelled_sentences\labelled_amazon.txt",delimiter="\t", names=["Sentence", "Label"])

In [3]:
# check if data is balance in all three dataframes

# yelp
ones_yelp = len(yelp[yelp['Label'] == 1])
zeros_yelp = len(yelp[yelp['Label'] == 0])
print('Number of 1s in Yelp:', ones_yelp)
print('Number of 0s in Yelp:', zeros_yelp)

#imdb
ones_imdb = len(imdb[imdb['Label'] == 1])
zeros_imdb = len(imdb[imdb['Label'] == 0])
print('Number of 1s in Imdb:', ones_imdb)
print('Number of 0s in Imdb:', zeros_imdb)

#amazon
ones_amazon = len(amazon[amazon['Label'] == 1])
zeros_amazon = len(amazon[amazon['Label'] == 0])
print('Number of 1s in Amazon:', ones_amazon)
print('Number of 0s in Amazon:', zeros_amazon)

Number of 1s in Yelp: 500
Number of 0s in Yelp: 500
Number of 1s in Imdb: 386
Number of 0s in Imdb: 362
Number of 1s in Amazon: 500
Number of 0s in Amazon: 500


The data in the Yelp and Amazon files is balanced because there are the same number of 1s and 0s as labels. 
The data in the Imdb file can be considered almost balancen because the number of 1s and 0s is almost the same (386 and 362, respectively). The ratio of 1s to 0s is 386/362 = 1.067.

#### b) Pre-processing datasets

In [4]:
def preprocessing(data):
    stop_words = set(stopwords.words('english')) # find stop words in English language
    lemmatizer = WordNetLemmatizer() # declare nltk lemmatizer

    # iterate through every sentence and replace it by itself lemmatized, without punctuation and without stop words
    for i in data['Sentence'].index:
    
        # remove punctuation
        sentence_no_punct = ''
        for char in (data.at[i, 'Sentence']):
            if char not in string.punctuation:
                sentence_no_punct = sentence_no_punct + char
        (data.at[i, 'Sentence']) = sentence_no_punct

        word_tokens = word_tokenize(data.at[i, 'Sentence'])
        # remove stop words and lemmatize
        word_tokens = [lemmatizer.lemmatize(word) for word in word_tokens if word not in stop_words]
        word_tokens = [lemmatizer.lemmatize(word, 'v') for word in word_tokens]
        word_tokens = [lemmatizer.lemmatize(word, 'a') for word in word_tokens]
        # remove conjunction words
        word_tokens = [word for word in word_tokens if word[-2:] != 'nt']
        (data.at[i, 'Sentence']) = ' '.join(word_tokens)
        
    return data

In [5]:
# convert all letters to lower case
yelp = yelp.applymap(lambda s:s.lower() if type(s) == str else s)
imdb = imdb.applymap(lambda s:s.lower() if type(s) == str else s)
amazon = amazon.applymap(lambda s:s.lower() if type(s) == str else s)

# lemmatize, remove punctuation, remove stop words
preprocessing(yelp)
preprocessing(imdb)
preprocessing(amazon);

For this part, we decided to convert all sentences to lower case, so that the same word with some upper case letters and without them would not be detected as different words since we are using the string type which takes into account their differences. 

We also stripped the sentences of stop words because they do not add any meaning as the same stop words appear in many different sentences.

Additionally, we removed the punctuation because it does not add any meaning to the word analysis exercise we will do in this question. 

We also lemmatized all the words because we are interested in knowing which class of words they belong to in order to understand the meaning of the sentence and not whether they are a noun, adjective, etc.

We finally removed words with the conjunction "n't" because these are also meaningless words like "isn't" and "didn't". They are basically stop words but are not caught by the stop word checker because of the extra "n't".

#### c) Split training and testing data

In [6]:
# split the three datasets into training and testing data according to the specifications

def split_data(data):
    data.reset_index(drop=True)
    training = (data.query('Label == 1' )).head(400).append((data.query('Label == 0' )).head(400))
    testing = (data.query('Label == 1' )).tail(100).append((data.query('Label == 0' )).tail(100))
    return training, testing

In [7]:
# split yelp
training_yelp, testing_yelp = split_data(yelp)

# split imdb
training_imdb, testing_imdb = split_data(imdb)

# split amazon
training_amazon, testing_amazon = split_data(amazon)

In [8]:
# concatenate training and testing data for all files
train_data = training_yelp.append(training_imdb, ignore_index=True).append(training_amazon, ignore_index=True)
print(np.shape(train_data)) # this not exactly 2400?

test_data = testing_yelp.append(testing_imdb, ignore_index=True).append(testing_amazon, ignore_index=True)
print(np.shape(test_data)) 

(2348, 2)
(600, 2)


#### d) Bag of Words model

For this question we cannot use the testing set to create the dictionary of unique words because the model needs to be created with the training set so that we can use the testing set as new data to test our model's ability to generalize. If we create the dictionary with the testing data, we are esentially using all the data as training data and would need to look for another set of new data to test the classifier.

In [15]:
# create set of unique words in training set
word_dictionary = {}

# iterate through every word or every sentence and store it in dictionary with count 0 (the count will be updated
# later when we iterate through both testing and training set
for i in train_data.index:
    word_tokens_training = word_tokenize(train_data.at[i, 'Sentence'])
    for word in word_tokens_training:
        if word not in word_dictionary.keys():
            word_dictionary[word] = 0
# count the number of occurences of each word in dictionary in training set
for i in train_data.index:
    word_tokens_training = word_tokenize(train_data.at[i, 'Sentence'])
    for word in word_tokens_training:
        if word in word_dictionary.keys():
            word_dictionary[word] += 1
            
# count the number of occurences of each word in dictionary in testing set  
for i in test_data.index:
    word_tokens_testing = word_tokenize(test_data.at[i, 'Sentence'])
    for word in word_tokens_testing:
        if word in word_dictionary.keys():
            word_dictionary[word] += 1
# create one feature vector per review
feature_column = [] # list to store the feature vectors and add to dataframe at the end
j = 0; # to indicate position in iterating through feature vector

for i in train_data.index:
    word_tokens_training = word_tokenize(train_data.at[i, 'Sentence'])
    feature_vector = [0] * len(word_dictionary.keys()) # to store feature vector in each iteration
    for dict_word in word_dictionary.keys():
        if dict_word in word_tokens_training:
            feature_vector[j] = word_dictionary[dict_word]
            j += 1
    feature_column.append(feature_vector) 
    j = 0
    
train_data['Vectors'] = feature_column

e) Pick postprocessing strategy

We will not apply mean substraction and division by standard deviation because this only makes sense if the data more or less follows a normal distribution. Since we cannot ensure this, we will choose some other method.

Also, the number of dimensions (words) of the feature vectors is significant, so choosing L1 or L2 norm might result in similar distance measures for sentences with different words, so we discarded these techniques. 

Log normalization can allow us to make data less skewed and focus on the relative differences rather than the absolute values, so we chose to apply this technique.

In [23]:
# log normalization implementation

log_norm = 0; # variable to store log norm value each iteration

for i in train_data['Vectors'].index:
    for element in train_data.at[i, 'Vectors']:
        #log_norm = math.log(element)
       
    

ValueError: math domain error

[92,
 7,
 13,
 13,
 1,
 1,
 1,
 3,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 